https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval.html

- Sentence Transformers as the embedding model

- Postgres as the vector store (we support many other vector stores too!)

- Llama 2 as the LLM (through llama.cpp)

In [1]:
# sentence transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

/Users/parkhyerin/Study/TIL_RAG/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from llama_index.llms.llama_cpp import LlamaCPP

model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

total size (MB): 7365.83


7025it [06:32, 17.90it/s]                          
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/parkhyerin/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32         

### Initialize Postgres

In [5]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "1234"
port = "5432"
user = "local"

conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [6]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

### Build an Ingestion Pipeline from Scratch

#### 1. Load Data

In [7]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [8]:
loader = PyMuPDFReader()
documents = loader.load(file_path="../data/llama2.pdf")

#### 2. Use a Text Splitter to Split Documents

In [9]:
from llama_index.core.node_parser import SentenceSplitter

In [10]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [11]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

#### 3. Manually Construct Nodes from Text Chunks

In [12]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

#### 4. Generate Embeddings for each Node

In [13]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

#### 5. Load Nodes into a Vector Store

In [14]:
vector_store.add(nodes)

['2b88adf2-1e68-4f8c-b4f7-d6675d87c573',
 '4a4a5bc2-6b23-4c88-bb57-566f95530cfd',
 '79e4c784-97aa-4bc6-9fb0-da560eba3499',
 'f89c5f8e-c728-40a8-af09-0c9234d1e0a0',
 '6a746525-023b-44e9-8ad7-dba8f8f0b6a8',
 'd38e15c2-505b-48ab-8cee-fe36266b0216',
 '9a8dcec1-7232-45d8-8c5f-d3cc17b36f8b',
 'a63bd2bc-6a19-4218-ae0c-0788d1a3d5bf',
 '691956e8-301a-4b37-ae4a-4926fe15f0d8',
 '387d2750-2299-49a6-a442-5f554578614a',
 '5a93ed07-663e-4e0e-b7a4-d8a6e965812d',
 '5ab07e8e-0617-431e-9868-b2ffe8c2604e',
 '886912e9-5300-49ce-8b99-4f709f08c8d1',
 'e2d9d25f-9443-4cfa-8a6f-fe62f1bbd4f0',
 'a8409e1c-7381-4770-9852-56b013163941',
 '24cabc8d-95ff-4c7c-af7d-4d1cff6c1cc2',
 '7661174a-1a7e-42eb-b5f2-36625dfb766f',
 '0b38c271-19eb-497d-912d-1920d40f7498',
 'b37861f8-06e8-4c94-982b-01a0a93c47a8',
 '9036a2aa-f94d-4503-a46d-244cfbb89bdd',
 'cf1c9d0e-0c2b-4d31-9a4b-da831df25449',
 'b72a090b-4dda-42cf-b74b-d739e28ca16c',
 '7bb9a9d7-745c-4b0a-ac19-d57b4a1f400e',
 '13a83fa6-9d5a-43a0-a80b-5cb969eedddf',
 'e505b518-8054-

### Build Retrieval Pipeline from Scratch

In [15]:
query_str = "Can you tell me about the key concepts for safety finetuning"

#### 1. Generate a Query Embedding

In [16]:
query_embedding = embed_model.get_query_embedding(query_str)

#### 2. Query the Vector Database

In [17]:
# construct vector store query
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [18]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

#### 3. Parse Result into a Set of Nodes

In [19]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

### 4. Put into a Retriever

In [20]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [21]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

### Plug this into our RetrieverQueryEngine to synthesize a response

In [22]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [23]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)


llama_print_timings:        load time =   26816.16 ms
llama_print_timings:      sample time =       4.96 ms /    56 runs   (    0.09 ms per token, 11290.32 tokens per second)
llama_print_timings: prompt eval time =   68618.51 ms /  1850 tokens (   37.09 ms per token,    26.96 tokens per second)
llama_print_timings:        eval time =    6749.99 ms /    55 runs   (  122.73 ms per token,     8.15 tokens per second)
llama_print_timings:       total time =   75461.54 ms /  1905 tokens


In [24]:
print(str(response))

 Based on the context information provided, Llama 2 performs better than other open-source models on most academic benchmarks, with the exception of coding benchmarks where there is a significant gap compared to GPT-3.5 and PaLM-2-L.


In [25]:
print(response.source_nodes[0].get_content())

Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard (3-shot)
–
–
52.3
65.7
51.2
Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4
are from OpenAI